In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

import warnings
from sklearn.exceptions import DataConversionWarning

# Ignore all warnings from scikit-learn
warnings.filterwarnings("ignore", module="sklearn")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
# Add more categories if needed



In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 10,
    "exp_name": ct.generate_readable_short_id("full_run_glogal_noise"), # full_run_glogal_noise
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 5,
    "plot": False,
    "n_jobs": -1,

    "calib_methods": [
                      "RF_d",
                      "RF_opt", 
                      "Platt", "ISO", "Beta", "VA",
                      "CT", "PPA",
                      "Shaker",
                      ],

    "calib_method_colors": {
        "RF_d": "blue", 
        "RF_opt": "orange", 
        "Platt": "Brown", 
        "ISO": "purple", 
        "Beta": "magenta", 
        "VA": "gray",
        "CT": "slategray",
        "PPA": "olive", 
        "Shaker": "gold",
    },


    "metrics": ["acc", "brier", "ece", "logloss", "time"],

    # calib param   
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth":  np.arange(2, 100).tolist() + [None], # [None, 5, 10, 15, 20, 30, 50], #
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf":  np.arange(1, 11).tolist(),
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False],
                    "laplace": [0,1]
                    },
    
    "oob": False,
    "laplace":0, # if set to 1, OOB must be false
    "curt_v": np.arange(1, 100).tolist(),

    "exp_key": "data_name",
    "exp_values": [
                  "cm1",
                  "datatrieve",
                  "kc1_class_level_defectiveornot",
                  "kc1",
                  "kc2",
                  "kc3",
                  "pc1",
                  "spect",
                  "spectf",
                  "vertebral", 
                  "wilt",
                  "parkinsons", 
                  "heart",
                  "wdbc",
                  "bank", 
                  "ionosphere", 
                  "HRCompetencyScores",
                  "spambase", 
                  "QSAR", 
                  "diabetes", 
                  "breast", 
                  "SPF",
                  "hillvalley",
                  "pc4",
                  "scene",
                  "Sonar_Mine_Rock_Data",
                  "Customer_Churn",
                  "jm1",
                  "eeg",
                  "phoneme",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param cm1 done
exp_param datatrieve done


/home/ra38jof/projects/RFC/.venv/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.001000095494077554] before, using random point [0.13706887488680972]
  warnings.warn(
/home/ra38jof/projects/RFC/.venv/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.001000237500121655] before, using random point [0.018956091910972653]
  warnings.warn(
/home/ra38jof/projects/RFC/.venv/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.0010007332941835772] before, using random point [0.00589496677320964]
  warnings.warn(
/home/ra38jof/projects/RFC/.venv/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.0010007332941835772] before, using random point [0.00589496677320964]
  warnings.warn(


exp_param kc1_class_level_defectiveornot done
exp_param kc1 done
exp_param kc2 done
exp_param kc3 done
exp_param pc1 done
exp_param spect done
exp_param spectf done
exp_param vertebral done
exp_param wilt done
exp_param parkinsons done
exp_param heart done
exp_param wdbc done
exp_param bank done
exp_param ionosphere done
exp_param HRCompetencyScores done
exp_param spambase done
exp_param QSAR done
exp_param diabetes done
exp_param breast done
exp_param SPF done
exp_param hillvalley done
exp_param pc4 done
exp_param scene done


/home/ra38jof/projects/RFC/.venv/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.001000095494077554] before, using random point [0.13706887488680972]
  warnings.warn(
/home/ra38jof/projects/RFC/.venv/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.001000237500121655] before, using random point [0.018956091910972653]
  warnings.warn(


exp_param Sonar_Mine_Rock_Data done
exp_param Customer_Churn done
exp_param jm1 done
exp_param eeg done
exp_param phoneme done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [5]:
ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}", colors=params["calib_method_colors"])

metric acc
Friedman Test Statistic: 61.679955081414946
P-value: 2.1799425037535008e-10
The differences between groups are significant.
metric brier
Friedman Test Statistic: 55.626666666666665
P-value: 3.3346846107677903e-09
The differences between groups are significant.
metric ece
Friedman Test Statistic: 58.39999999999998
P-value: 9.5924231986145e-10
The differences between groups are significant.
metric logloss
Friedman Test Statistic: 157.72444444444454
P-value: 4.782455245710116e-30
The differences between groups are significant.
metric time
Friedman Test Statistic: 216.55111111111114
P-value: 2.060873224013514e-42
The differences between groups are significant.


In [6]:
ct.save_metrics_to_latex(tables, params["metrics"], params['exp_name'])

In [7]:
calib_results_dict.keys()

dict_keys(['cm1_RF_d_runtime', 'cm1_RF_opt_runtime', 'cm1_Platt_runtime', 'cm1_ISO_runtime', 'cm1_PPA_runtime', 'cm1_Shaker_runtime', 'cm1_VA_runtime', 'cm1_Beta_runtime', 'cm1_CT_runtime', 'cm1_RF_d_time', 'cm1_RF_opt_time', 'cm1_Platt_time', 'cm1_ISO_time', 'cm1_Beta_time', 'cm1_VA_time', 'cm1_CT_time', 'cm1_PPA_time', 'cm1_Shaker_time', 'cm1_RF_d_acc', 'cm1_RF_opt_acc', 'cm1_Platt_acc', 'cm1_ISO_acc', 'cm1_Beta_acc', 'cm1_VA_acc', 'cm1_CT_acc', 'cm1_PPA_acc', 'cm1_Shaker_acc', 'cm1_RF_d_ece', 'cm1_RF_opt_ece', 'cm1_Platt_ece', 'cm1_ISO_ece', 'cm1_Beta_ece', 'cm1_VA_ece', 'cm1_CT_ece', 'cm1_PPA_ece', 'cm1_Shaker_ece', 'cm1_RF_d_brier', 'cm1_RF_opt_brier', 'cm1_Platt_brier', 'cm1_ISO_brier', 'cm1_Beta_brier', 'cm1_VA_brier', 'cm1_CT_brier', 'cm1_PPA_brier', 'cm1_Shaker_brier', 'cm1_RF_d_logloss', 'cm1_RF_opt_logloss', 'cm1_Platt_logloss', 'cm1_ISO_logloss', 'cm1_Beta_logloss', 'cm1_VA_logloss', 'cm1_CT_logloss', 'cm1_PPA_logloss', 'cm1_Shaker_logloss', 'datatrieve_RF_d_runtime', 'data